# Efficient Fully Homomorphic Encryption from Standard LWE Assumption
By [Zvika Brakerski and Vinod Vaikuntanathan](https://eprint.iacr.org/2011/344) in FOCS11

In [22]:
class DualMaskENC:
    def __init__(self, q: int, n : int):
        self._q = q
        self._n = n
        self._ring = IntegerModRing(self._q)
        self._secret = random_vector(self._ring, self._n)
        self._chi = GeneralDiscreteDistribution(range(int(sqrt(q))))
        
        
    def enc(self, m : int):
        assert m == 0 or m == 1
        a = random_vector(self._ring, self._n)
        e = self._chi.get_random_element()
        return (a, a*self._secret + 2*e + m)
    
    def dec(self, a, b):
        return int(b - a*self._secret) % 2
    
    def xor(self, a1, b1, a2, b2):
        return (a1 + a2, b1 + b2)
    
    def and_(self, a1, b1, a2, b2):
        pass
    
    
class SHKey:
    def __init__(self, N:int, M:int, Q:int, L:int):
        self._Ring = IntegerModRing(Q)
        self._L = L 
        self._N = N
        self._Q = Q
        self._logQ = len(Q.bits())
        self._clamp_ring = IntegerModRing(int(sqrt(self._Q))+1)
        
        assert N*self._logQ < M
        
        self._M = M
        self._S = list()
        
        for x in range(L+1):
            v = random_vector(self._Ring, N)
            if x < L:
                v[0] = 1
            self._S.append(v)
        
        self._psi = list()
        for l in range(L):
            self._psi.append(list())
            for i in range(N):
                self._psi[l].append(list())
                for j in range(N):
                    self._psi[l][i].append(list())
                    for tau in range(self._logQ):
                        self._psi[l][i][j].append(list())
                        a = random_vector(self._Ring, self._N)
                        b = random_vector(self._Ring, 1)
                        err = (a*self._S[l+1]) + 2*b[0] + \
                                (2^tau) * self._S[l][i] * self._S[l][j]
                        self._psi[l][i][j][tau] = (a, err)
                        
    def eval_key(self):
        return self._psi
    
    def sk(self):
        return self._S[self._L]
    
    def pk(self):
        A=random_matrix(self._Ring, self._M, self._N)
        e_tmp=random_vector(self._clamp_ring, self._M)
        e = vector(self._Ring, [self._Ring(int(x)) for x in e_tmp])
        return (A, A*self._S[0] + 2*e)
    
    def __repr__(self):
        return "SKs=\n{}\npk=\n{}".format(self._S, self.pk())
    
    
    
class SH:
    def __init__(self, kappa : int, epsilon : float):
        self._epsilon = epsilon
        self._kappa = kappa
        self._n = kappa+1
        rand_min = int(float(2)^(float(self._n)^epsilon))
        rand_max = 2*rand_min
        self._q = random_prime(rand_max, lbound=rand_min)
        assert rand_min <= self._q and rand_max >= self._q
        self._L = int(epsilon* len(self._n.bits()))
        self._m = len(self._q.bits())*self._n + 2*kappa + 1
            
    @classmethod
    def from_params(klass, kappa:int, epsilon:float, n:int, q:int):
        nc = klass.__new__(klass)
        nc._epsilon = epsilon
        nc._kappa = kappa
        nc._n = n
        nc._q = q
        nc._L = int(epsilon* len(nc._n.bits()))
        nc._m = len(nc._q.bits())*nc._n + 2*kappa + 1
        return nc
    
    def key_gen(self):
        key = SHKey(self._n, self._m, self._q, self._L)
        return key
    
    def __repr__(self):
        return "ϵ={}, κ={}, n={}, m={}, q={}, L={}".format(self._epsilon, 
                                                         self._kappa, 
                                                         self._n,
                                                         self._m,
                                                         self._q,
                                                         self._L)

sh = SH.from_params(32, 0.7, 33, 3691); print(sh)
kg=sh.key_gen(); print(kg)

ϵ=0.700000000000000, κ=32, n=33, m=461, q=3691, L=4
SKs=
[(1, 1784, 1671, 567, 648, 1521, 2181, 262, 1027, 2661, 1653, 2159, 1175, 225, 2240, 2001, 1689, 3212, 2459, 3056, 1509, 1069, 1884, 2965, 462, 1077, 1477, 2599, 1073, 1945, 2423, 487, 1010), (1, 2616, 3250, 3379, 944, 3092, 2885, 3202, 2841, 271, 2876, 144, 960, 654, 3671, 2928, 570, 3647, 3222, 2174, 230, 1061, 2793, 2742, 971, 3554, 3683, 777, 783, 3672, 141, 2990, 2441), (1, 2742, 1310, 1004, 1443, 952, 2774, 647, 2197, 1888, 1306, 1071, 2502, 702, 1495, 2536, 2165, 2440, 2721, 1141, 1083, 2716, 1591, 1822, 1862, 1944, 1711, 3425, 108, 524, 3630, 2385, 2928), (1, 181, 3471, 474, 2413, 3606, 1329, 584, 465, 2100, 453, 2776, 1854, 150, 1744, 2958, 3215, 2485, 2347, 418, 3667, 965, 2702, 712, 759, 1297, 642, 1076, 53, 3190, 2231, 2836, 157), (2759, 3042, 194, 885, 3687, 40, 3026, 1750, 1357, 309, 536, 709, 2153, 1539, 3309, 1750, 3597, 2986, 358, 3283, 2875, 3106, 3610, 447, 3034, 1895, 1586, 1329, 2466, 2254, 350, 26, 1692)]
pk